# 1 
Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо нейронной сети, работающей airline-passengers (она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить её точность. Приложите анализ


# 2
Предложите свои варианты решения проблемы исчезающего градиента в RNN

# 2
Для решения проблемы исчезающего градиента в rnn можно в качестве функции активации на пропускающих воротах взять relu вместо sigma так же можно следущее за ним операцию умножения заменить на сложении более подробно описано https://arxiv.org/abs/2308.05629
Данное использование в отличие от lSTM и GRU менее операционно затратно

In [3]:
%%time
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.layers import Dropout, BatchNormalization



CPU times: user 29 µs, sys: 5 µs, total: 34 µs
Wall time: 39.1 µs


## Вариант по умолчанию

In [4]:
%%time
max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 128 # увеличьте значение для ускорения обучения


from keras.backend import dropout
def train_nn_1():
    print('Загрузка данных...')
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    print(len(x_train), 'тренировочные последовательности')
    print(len(x_test), 'тестовые последовательности')
  
    print('Pad последовательности (примеров в x единицу времени)')
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
  
    print('Построение модели...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='LeakyReLU'))            # New line
    model.add(Dropout(0.4))
    model.add(Dense(16, activation='elu'))
    model.add(Dropout(0.2))                                  # New line
    model.add(Dense(1, activation='sigmoid'))
  
    # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
    model.compile(loss='binary_crossentropy',
                  optimizer='RMSprop', # при использовании этого оптимайзера модель показывает наилучшие результаты.
                  metrics=['accuracy'])
    callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
    print('Процесс обучения...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10, # увеличьте при необходимости
              callbacks = callbacks,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)
train_nn_1()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Построение модели...
Процесс обучения...
Epoch 1/10
196/196 [==============================] - 104s 480ms/step - loss: 0.5884 - accuracy: 0.6757 - val_loss: 0.3852 - val_accuracy: 0.8264
Epoch 2/10


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


196/196 [==============================] - 81s 413ms/step - loss: 0.3999 - accuracy: 0.8355 - val_loss: 0.4639 - val_accuracy: 0.7989
Epoch 3/10
196/196 [==============================] - 78s 398ms/step - loss: 0.3356 - accuracy: 0.8646 - val_loss: 0.3488 - val_accuracy: 0.8476
Epoch 4/10
196/196 [==============================] - 76s 389ms/step - loss: 0.2986 - accuracy: 0.8833 - val_loss: 0.3981 - val_accuracy: 0.8428
Epoch 5/10
196/196 [==============================] - 75s 382ms/step - loss: 0.2720 - accuracy: 0.8967 - val_loss: 0.3481 - val_accuracy: 0.8525
Epoch 6/10
196/196 [==============================] - 74s 380ms/step - loss: 0.2550 - accuracy: 0.9033 - val_loss: 0.5029 - val_accuracy: 0.8420
Epoch 7/10
196/196 [==============================] - 74s 378ms/step - loss: 0.2367 - accuracy: 0.9118 - val_loss: 0.4619 - val_accuracy: 0.8291
Epoch 8/10
196/196 [==============================] - 75s 383ms/step - loss: 0.2131 - accuracy: 0.9214 - val_loss: 0.3792 - val_accuracy: 0.8

## Увеличим количество эпох 
## Добавим нормализацию после ккаждого слоя и увеличим dropout для уменьшения переобучения 

In [5]:
%%time
max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 168 # увеличьте значение для ускорения обучения
from keras.backend import dropout
def train_nn_2():
    print('Загрузка данных...')
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    print(len(x_train), 'тренировочные последовательности')
    print(len(x_test), 'тестовые последовательности')
  
    print('Pad последовательности (примеров в x единицу времени)')
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
  
    print('Построение модели...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))
    model.add(Dropout(0.45))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))            # New line
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='LeakyReLU'))
    model.add(Dropout(0.3))                                  # New line
    model.add(Dense(1, activation='sigmoid'))
  
    # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
    model.compile(loss='binary_crossentropy',
                  optimizer='RMSprop', # при использовании этого оптимайзера модель показывает наилучшие результаты.
                  metrics=['accuracy'])
    callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
    print('Процесс обучения...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15, # увеличьте при необходимости
              callbacks = callbacks,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)
train_nn_2()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Построение модели...
Процесс обучения...
Epoch 1/15
149/149 [==============================] - 79s 495ms/step - loss: 0.5527 - accuracy: 0.7070 - val_loss: 0.6433 - val_accuracy: 0.5243
Epoch 2/15
149/149 [==============================] - 66s 445ms/step - loss: 0.3666 - accuracy: 0.8468 - val_loss: 0.4629 - val_accuracy: 0.8117
Epoch 3/15
149/149 [==============================] - 63s 424ms/step - loss: 0.3199 - accuracy: 0.8683 - val_loss: 0.3655 - val_accuracy: 0.8354
Epoch 4/15
149/149 [==============================] - 60s 400ms/step - loss: 0.2889 - accuracy: 0.8841 - val_loss: 0.3747 - val_accuracy: 0.8277
Epoch 5/15
149/149 [==============================] - 59s 396ms/step - loss: 0.2617 - accuracy: 0.8978 - val_loss: 0.3409 - val_accuracy: 0.8502
Epoch 6/15
149/149 [=============


## Будем использовать одинаковые функции активации а так же добавим градиентное понижение шага при градиентном спуске

In [7]:
%%time
max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 168 # увеличьте значение для ускорения обучения
from keras.backend import dropout
def train_nn_2():
    print('Загрузка данных...')
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    print(len(x_train), 'тренировочные последовательности')
    print(len(x_test), 'тестовые последовательности')
  
    print('Pad последовательности (примеров в x единицу времени)')
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
  
    print('Построение модели...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))
    model.add(Dropout(0.45))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))            # New line
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='LeakyReLU'))
    model.add(Dropout(0.4))                                  # New line
    model.add(Dense(1, activation='sigmoid'))
    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)
  
    # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizer,
                  metrics=['accuracy']) 
    callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
    print('Процесс обучения...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15, # увеличьте при необходимости
              callbacks = callbacks,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)
train_nn_2()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Построение модели...
Процесс обучения...
Epoch 1/15
149/149 [==============================] - 73s 457ms/step - loss: 0.6875 - accuracy: 0.5386 - val_loss: 0.6440 - val_accuracy: 0.6086
Epoch 2/15
149/149 [==============================] - 61s 412ms/step - loss: 0.4511 - accuracy: 0.7953 - val_loss: 0.5181 - val_accuracy: 0.7729
Epoch 3/15
149/149 [==============================] - 59s 393ms/step - loss: 0.3132 - accuracy: 0.8750 - val_loss: 0.3312 - val_accuracy: 0.8574
Epoch 4/15
149/149 [==============================] - 59s 394ms/step - loss: 0.2466 - accuracy: 0.9070 - val_loss: 0.4377 - val_accuracy: 0.8570
Epoch 5/15
149/149 [==============================] - 58s 387ms/step - loss: 0.2029 - accuracy: 0.9262 - val_loss: 0.4465 - val_accuracy: 0.8428
Epoch 6/15
149/149 [=============

## Уменьшим количество эпох до 4 на 4 жпохе как видим результат обучения только ухудшается

In [9]:
%%time
max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 180 # увеличьте значение для ускорения обучения
from keras.backend import dropout
def train_nn_3():
    print('Загрузка данных...')
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    print(len(x_train), 'тренировочные последовательности')
    print(len(x_test), 'тестовые последовательности')
  
    print('Pad последовательности (примеров в x единицу времени)')
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
  
    print('Построение модели...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))
    model.add(Dropout(0.45))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='LeakyReLU'))            # New line
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='LeakyReLU'))
    model.add(Dropout(0.4))                                  # New line
    model.add(Dense(1, activation='sigmoid'))
    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)
  
    # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
    model.compile(loss='binary_crossentropy',
                  optimizer= optimizer,
                  metrics=['accuracy']) 
    callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
    print('Процесс обучения...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=5, # увеличьте при необходимости
              callbacks = callbacks,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)
train_nn_3()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Построение модели...
Процесс обучения...
Epoch 1/5
139/139 [==============================] - 69s 459ms/step - loss: 0.7122 - accuracy: 0.4990 - val_loss: 0.6941 - val_accuracy: 0.4996
Epoch 2/5
139/139 [==============================] - 57s 413ms/step - loss: 0.6657 - accuracy: 0.5733 - val_loss: 1.2658 - val_accuracy: 0.5032
Epoch 3/5
139/139 [==============================] - 56s 403ms/step - loss: 0.4996 - accuracy: 0.7616 - val_loss: 0.3609 - val_accuracy: 0.8384
Epoch 4/5
139/139 [==============================] - 55s 396ms/step - loss: 0.3276 - accuracy: 0.8692 - val_loss: 0.3814 - val_accuracy: 0.8512
Epoch 5/5
139/139 [==============================] - 4s 27ms/step - loss: 0.4646 - accuracy: 0.8406
Результат при тестировании: 0.4645598530769348
Тестовая точность: 0.84064000844955

## Итог
Как видим уменьшение эпох позволило улучшить точность на валидационной выборке при дальнейшем обучении модель переобучается. Еще большей процент dropout не решает проблему увеличение бача тоже не особо помогает скорей всего данное состояние является хорошим локальным минимумумом.